In [50]:
import subprocess
from statistics import mean

default_problem = "101101011010110110011111011010100010110101111001000010011011101000"

def parse_output(output):
    output_array = output.strip().split(' ')
    score = output_array[1]
    time = output_array[2]
    return float(score), int(time)

def calculate_mean_score_and_time(solver, iterations = 100000, problem = default_problem):
    results = []
    command = ["./metaheuristics",
               "--solver", solver,
               "--problem", problem,
               "-i", str(iterations)]
    
    for i in range(5):
        results.append(parse_output(subprocess.run(command, capture_output=True, text=True).stdout))

    mean_score = mean([item[0] for item in results])
    mean_time = mean([item[1] for item in results])
    print(results)
    return mean_score, mean_time

print(calculate_mean_score_and_time("solve_tabu_avoid_snake"))

[(10.0, 5646853920), (10.0, 3596484612), (10.0, 3921364084), (10.0, 5941617532), (10.0, 3624500966)]
(10.0, 4546164222.8)
